**Introduction**
  We propose to achieve movie genre classification based only on movie poster images. A deep neural network is constructed to jointly describe visual appearance and object information, and classify a given movie poster image into genres. Because a movie may belong to multiple genres, this is a multi-label image classification problem. To facilitate related studies, we collect a large-scale movie poster dataset, associated with various metadata. Based on this dataset, we fine-tune a pre-trained convolutional neural network to extract visual representation and adopt a state-of-the-art framework to detect objects in posters. Two types of information are then integrated by the proposed neural network. In the evaluation, we show that the proposed method yields encouraging performance, which is much better than previous works. "

In [1]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 38kB/s 
     |████████████████████████████████| 3.8MB 43.7MB/s 
     |████████████████████████████████| 450kB 50.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


# New Section

# New Section

In [0]:
import pandas as pd
import json
import glob  
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D


In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [0]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [5]:
print(tf.__version__)

2.0.0


In [6]:
!git clone https://github.com/sweetcoonoor/Multiclassification_Genre.git

Cloning into 'Multiclassification_Genre'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 829 (delta 1), reused 0 (delta 0), pack-reused 822
Receiving objects: 100% (829/829), 24.18 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [0]:
!git clone https://github.com/laxmimerit/Movies-Poster_Dataset.git

Cloning into 'Movies-Poster_Dataset'...
remote: Enumerating objects: 7873, done.
remote: Counting objects: 100% (7873/7873), done.
remote: Compressing objects: 100% (7873/7873), done.
remote: Total 7873 (delta 1), reused 7872 (delta 0), pack-reused 0
Receiving objects: 100% (7873/7873), 246.59 MiB | 38.64 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (7871/7871), done.


In [0]:
ff=open("/content/Multiclassification_Genre/1980.txt","r")
Movies=ff.read()
comma=Movies.replace("}","},")
newc=comma[:-2]
sqaureAdd="["+newc+"]"
aaa=sqaureAdd.replace("\n","")
jj=aaa.replace("ObjectId(","").replace("\")","\"")
fin=json.loads(jj)
df = pd.DataFrame.from_dict(fin, orient='columns')

In [0]:
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
dfFin
dfFin2=dfFin

In [14]:
all_files = glob.glob("/content/Multiclassification_Genre/*.txt")

print((all_files))
count=0
for files in all_files:
  try:
    ff=open(files,"r",encoding='utf-16')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
  except:
    ff=open(files,"r",encoding='utf-8')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
    

['/content/Multiclassification_Genre/1994.txt', '/content/Multiclassification_Genre/1981.txt', '/content/Multiclassification_Genre/1999.txt', '/content/Multiclassification_Genre/2009.txt', '/content/Multiclassification_Genre/2010.txt', '/content/Multiclassification_Genre/1998.txt', '/content/Multiclassification_Genre/1996.txt', '/content/Multiclassification_Genre/1986.txt', '/content/Multiclassification_Genre/2011.txt', '/content/Multiclassification_Genre/1984.txt', '/content/Multiclassification_Genre/1982.txt', '/content/Multiclassification_Genre/2015.txt', '/content/Multiclassification_Genre/1980.txt', '/content/Multiclassification_Genre/1989.txt', '/content/Multiclassification_Genre/1992.txt', '/content/Multiclassification_Genre/2006.txt', '/content/Multiclassification_Genre/1991.txt', '/content/Multiclassification_Genre/1993.txt', '/content/Multiclassification_Genre/2008.txt', '/content/Multiclassification_Genre/1997.txt', '/content/Multiclassification_Genre/2013.txt', '/content/Mu

In [111]:
Movie_data=pd.concat([dfFin,dfFin2])
#Movie_data.head()
Movie_data


,Actors,Awards,Box_office,Country,Director,Genre,Language,Metascore,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,_id,imdbID,imdbRating,imdbVotes
0,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",Won 1 Oscar. Another 15 wins & 18 nominations.,"290,158,751",USA,Irvin Kershner,"Action, Adventure, Fantasy",English,79,http://ia.media-imdb.com/images/M/MV5BMjE2MzQw...,PG,20 Jun 1980,True,124 min,Star Wars: Episode V - The Empire Strikes Back,movie,"Leigh Brackett (screenplay), Lawrence Kasdan (...",56fb860e71a0651b0c306f55,tt0080684,8.8,"799,579"
1,"Gene Wilder, Richard Pryor, Georg Stanford Bro...",1 nomination.,"101,500,000",USA,Sidney Poitier,"Comedy, Crime",English,50,http://ia.media-imdb.com/images/M/MV5BMTM4OTI3...,R,12 Dec 1980,True,111 min,Stir Crazy,movie,Bruce Jay Friedman,56fb860e71a0651b0c306f56,tt0081562,6.8,"17,705"
2,"Kareem Abdul-Jabbar, Lloyd Bridges, Peter Grav...",Nominated for 1 Golden Globe. Another 2 wins &...,"83,400,000",USA,"Jim Abrahams, David Zucker, Jerry Zucker",Comedy,English,N/A,http://ia.media-imdb.com/images/M/MV5BNDU2MjE4...,PG,02 Jul 1980,True,88 min,Airplane!,movie,"Jim Abrahams (written for the screen by), Davi...",56fb860e71a0651b0c306f57,tt0080339,7.8,"154,994"
3,"Clint Eastwood, Sondra Locke, Geoffrey Lewis, ...",1 win.,None,USA,Buddy Van Horn,"Action, Comedy",English,N/A,http://ia.media-imdb.com/images/M/MV5BMTc5MTc5...,PG,17 Dec 1980,True,116 min,Any Which Way You Can,movie,"Jeremy Joe Kronsberg (characters), Stanford Sh...",56fb860e71a0651b0c306f58,tt0080377,6.0,"12,639"
4,"Goldie Hawn, Eileen Brennan, Armand Assante, R...",Nominated for 3 Oscars. Another 1 win & 3 nomi...,"69,800,000",USA,Howard Zieff,"Comedy, War",English,N/A,http://ia.media-imdb.com/images/M/MV5BMjIyNjkx...,R,10 Oct 1980,True,109 min,Private Benjamin,movie,"Nancy Meyers, Charles Shyer, Harvey Miller",56fb860e71a0651b0c306f59,tt0081375,6.1,"17,591"
5,"Sissy Spacek, Tommy Lee Jones, Levon Helm, Phy...",Won 1 Oscar. Another 8 wins & 13 nominations.,"79,900,000",USA,Michael Apted,"Biography, Drama, Music",English,N/A,http://ia.media-imdb.com/images/M/MV5BMTkwMjUx...,PG,07 Mar 1980,True,124 min,Coal Miner's Daughter,movie,"Thomas Rickman (screenplay), Loretta Lynn (aut...",56fb860e71a0651b0c306f5a,tt0080549,7.5,"12,692"
6,"Burt Reynolds, Jackie Gleason, Jerry Reed, Dom...",1 win & 4 nominations.,"66,100,000",USA,Hal Needham,"Action, Comedy","English, Spanish",N/A,http://ia.media-imdb.com/images/M/MV5BMTc3MjUw...,PG,15 Aug 1980,True,100 min,Smokey and the Bandit II,movie,"Hal Needham (characters created by), Robert L....",56fb860e71a0651b0c306f5b,tt0081529,5.1,"10,818"
7,"Brooke Shields, Christopher Atkins, Leo McKern...",Nominated for 1 Oscar. Another 3 wins & 7 nomi...,None,USA,Randal Kleiser,"Adventure, Drama, Romance",English,N/A,http://ia.media-imdb.com/images/M/MV5BMTQwNTM5...,R,02 Jul 1980,True,104 min,The Blue Lagoon,movie,"Henry De Vere Stacpoole (novel), Douglas Day S...",56fb860e71a0651b0c306f5c,tt0080453,5.7,"48,868"
8,"John Belushi, Dan Aykroyd, James Brown, Cab Ca...",1 win.,"54,200,000",USA,John Landis,"Action, Comedy, Crime",English,N/A,http://ia.media-imdb.com/images/M/MV5BMTM1Mjk3...,R,20 Jun 1980,True,133 min,The Blues Brothers,movie,"Dan Aykroyd, John Landis",56fb860e71a0651b0c306f5d,tt0080455,7.9,"138,196"
9,"Donald Sutherland, Mary Tyler Moore, Judd Hirs...",Won 4 Oscars. Another 15 wins & 14 nominations.,"54,800,000",USA,Robert Redford,Drama,English,85,http://ia.media-imdb.com/images/M/MV5BNTQwNTc4...,R,27 Feb 1981,True,124 min,Ordinary People,movie,"Judith Guest (novel), Alvin Sargent (screenplay)",56fb860e71a0651b0c306f5e,tt0081283,7.8,"34,021"


Data Exploration and Pre-processing

In [0]:
Movie_Final=Movie_data[['imdbID','Title','Genre',]]
Movie_Final

In [0]:
Movie_Final.to_csv('Movie_genre.csv',encoding="ISO-8859-1")




In [108]:
#Onehot encoding
data = pd.read_csv("Movie_genre.csv")
data.shape

#df.dropna(subset=['Genre'],inplace=True)
#df.drop_duplicates(subset=['imdbID'], keep='first', inplace=True)
#df.set_index('imdbID',inplace=True)
#target = df['Genre'].apply(lambda x: [w.strip() for w in x.split(',')])
#genres = pd.get_dummies(target.apply(pd.Series).stack()).sum(level=0)


(6984, 4)

In [109]:
data

,Unnamed: 0,imdbID,Title,Genre
0,0,tt0080684,Star Wars: Episode V - The Empire Strikes Back,A
1,1,tt0081562,Stir Crazy,C
2,2,tt0080339,Airplane!,C
3,3,tt0080377,Any Which Way You Can,A
4,4,tt0081375,Private Benjamin,C
5,5,tt0080549,Coal Miner's Daughter,B
6,6,tt0081529,Smokey and the Bandit II,A
7,7,tt0080453,The Blue Lagoon,A
8,8,tt0080455,The Blues Brothers,A
9,9,tt0081283,Ordinary People,D


In [117]:
Movie_genre=Movie_Final
Movie_genre['Genre']=[elem[0] for elem in Movie_genre['Genre']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [115]:
Movie_genre['Genre']

0     A
1     C
2     C
3     A
4     C
5     B
6     A
7     A
8     A
9     D
10    A
11    D
12    D
13    C
14    C
15    C
16    D
17    C
18    M
19    D
20    A
21    B
22    A
23    B
24    F
25    C
26    C
27    F
28    H
29    C
     ..
67    H
68    C
69    C
70    C
71    C
72    C
73    D
74    D
75    D
76    A
77    A
78    F
79    C
80    B
81    C
82    C
83    D
84    C
85    A
86    D
87    C
88    B
89    D
90    D
91    C
92    D
93    R
94    C
95    D
96    D
Name: Genre, Length: 6984, dtype: object

In [0]:
Moviegenrestats=Movie_genre.groupby('Genre').size().reset_index(name='counts')

In [0]:
Moviegenrestats=Moviegenrestats.sort_values('counts',ascending=False)

In [0]:
MovieFilter=list(Moviegenrestats)

In [0]:
#Getting the image Id's
image_paths = glob.glob("/content/Multiclassification_Genre/images/*.jpg")
image_ids = []
for path in image_paths:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [89]:
traindatagenerator=train_datagen.flow_from_dataframe(
    dataframe=Movie_Final,
    directory='/content/Multiclassification_Genre/images',
    x_col='imdbID',
    y_col='Genre',
    batch_size=50,
    shuffle=True)

Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 6984 invalid image filename(s) in x_col="imdbID". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [91]:
testdatagenerator=test_datagen.flow_from_dataframe(
    dataframe=testdatagen,
    directory='/content/Multiclassification_Genre/images',
    x_col='imdbID',
    y_col='Genre',
    batch_size=50,
    shuffle=True
    )

NameError: ignored